In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Precursor-steps" data-toc-modified-id="Precursor-steps-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Precursor steps</a></span></li><li><span><a href="#Step-1:-Make-transcribed-lexicons" data-toc-modified-id="Step-1:-Make-transcribed-lexicons-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Step 1: Make transcribed lexicons</a></span><ul class="toc-item"><li><span><a href="#Check-for-transcribed-lexicons" data-toc-modified-id="Check-for-transcribed-lexicons-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Check for transcribed lexicons</a></span></li><li><span><a href="#How-are-transcribed-lexicon-relations-made?" data-toc-modified-id="How-are-transcribed-lexicon-relations-made?-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>How are transcribed lexicon relations made?</a></span></li></ul></li><li><span><a href="#Step-2:-Segment-inventory-alignment" data-toc-modified-id="Step-2:-Segment-inventory-alignment-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Step 2: Segment inventory alignment</a></span><ul class="toc-item"><li><span><a href="#Step-2a:-Define-inventory-alignment-projections" data-toc-modified-id="Step-2a:-Define-inventory-alignment-projections-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Step 2a: Define inventory alignment projections</a></span></li><li><span><a href="#Step-2b:-Apply-inventory-alignment-projections" data-toc-modified-id="Step-2b:-Apply-inventory-alignment-projections-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Step 2b: Apply inventory alignment projections</a></span><ul class="toc-item"><li><span><a href="#Check-for-projection-definitions" data-toc-modified-id="Check-for-projection-definitions-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Check for projection definitions</a></span></li><li><span><a href="#How-are-inventory-alignment-projections-actually-applied?" data-toc-modified-id="How-are-inventory-alignment-projections-actually-applied?-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>How are inventory alignment projections actually applied?</a></span></li><li><span><a href="#Apply-projection-definitions" data-toc-modified-id="Apply-projection-definitions-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Apply projection definitions</a></span></li></ul></li></ul></li><li><span><a href="#Step-3:-Generating-channel-and-lexicon-distributions" data-toc-modified-id="Step-3:-Generating-channel-and-lexicon-distributions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Step 3: Generating channel and lexicon distributions</a></span><ul class="toc-item"><li><span><a href="#Step-3a:-Generating-channel-distributions-and-associated-metadata" data-toc-modified-id="Step-3a:-Generating-channel-distributions-and-associated-metadata-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Step 3a: Generating channel distributions and associated metadata</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Channel-distributions" data-toc-modified-id="Channel-distributions-6.1.2"><span class="toc-item-num">6.1.2&nbsp;&nbsp;</span>Channel distributions</a></span></li></ul></li><li><span><a href="#Step-3b:-Filtering-and-alignment-('vocabulary-normalization')" data-toc-modified-id="Step-3b:-Filtering-and-alignment-('vocabulary-normalization')-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Step 3b: Filtering and alignment ('vocabulary normalization')</a></span><ul class="toc-item"><li><span><a href="#Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution" data-toc-modified-id="Filter-transcription-lexicons-to-only-include-words-that-can-be-modeled-by-a-given-channel-distribution-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Filter transcription lexicons to only include words that can be modeled by a given channel distribution</a></span></li><li><span><a href="#Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary" data-toc-modified-id="Filter-transcription-lexicons-to-only-include-words-that-are-in-a-language-model's-vocabulary-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Filter transcription lexicons to only include words that are in a language model's vocabulary</a></span></li><li><span><a href="#Filter-the-conditioning-events-of-channel-distributions-to-only-include-$n$-phones-contained-in-a-transcription-lexicon's-segmental-wordform-list" data-toc-modified-id="Filter-the-conditioning-events-of-channel-distributions-to-only-include-$n$-phones-contained-in-a-transcription-lexicon's-segmental-wordform-list-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Filter the conditioning events of channel distributions to only include $n$-phones contained in a transcription lexicon's segmental wordform list</a></span></li></ul></li><li><span><a href="#Step-3c:-Define-conditional-distributions-on-segmental-wordforms-given-an-orthographic-wordform" data-toc-modified-id="Step-3c:-Define-conditional-distributions-on-segmental-wordforms-given-an-orthographic-wordform-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Step 3c: Define conditional distributions on segmental wordforms given an orthographic wordform</a></span></li><li><span><a href="#Step-3d:-Define-a-distribution-over-the-transcribed-vocabulary-of-a-language-model-for-each-n-gram-context-in-a-set-of-(possibly-empty)-n-gram-contexts" data-toc-modified-id="Step-3d:-Define-a-distribution-over-the-transcribed-vocabulary-of-a-language-model-for-each-n-gram-context-in-a-set-of-(possibly-empty)-n-gram-contexts-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Step 3d: Define a distribution over the transcribed vocabulary of a language model for each n-gram context in a set of (possibly empty) n-gram contexts</a></span></li></ul></li><li><span><a href="#Step-4:-Generating-posterior-distributions" data-toc-modified-id="Step-4:-Generating-posterior-distributions-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Step 4: Generating posterior distributions</a></span></li><li><span><a href="#Step-5:-Generating-analysis-measures" data-toc-modified-id="Step-5:-Generating-analysis-measures-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Step 5: Generating analysis measures</a></span></li></ul></div>

# Overview

This notebook describes the processing pipeline from 
 - gating data
 - transcribed lexicon
 - a language model and (possibly empty) n-gram contexts

to 
 - channel distribution
 - lexicon distribution(s) (distributions over wordforms)
 - expected posterior distribution over intended wordform given what has been produced of what was intended.
 
 
It describes what happens at each step, checks some pre- and post-conditions, describes what you, the user must do (if anything), and scripts some commands to automatically do the necessary processing.

# Imports

In [2]:
import papermill as pm

In [3]:
from tqdm import tqdm

In [4]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

import json
import csv

In [5]:
from collections import OrderedDict

In [6]:
repo_dir = getcwd()
repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [7]:
repo_contents_0 = listdir()
repo_contents_0

['boilerplate.py',
 'LTR_Buckeye',
 '.gitignore',
 '__pycache__',
 '1 initial directory setup.txt',
 '2a alignment_paths_and_cmds.sh',
 'Align transcriptions.ipynb',
 'LTR_CMU_destressed',
 'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
 'Processing Driver Notebook.ipynb',
 'old',
 'GD_AmE',
 '.ipynb_checkpoints',
 'string_utils.py',
 'LTR_CMU_stressed',
 '.git',
 'LTR_newdic_destressed']

# Precursor steps

 - I assume all relevant transcriptions have been converted to Unicode IPA characters and that segment sequences have `.` as a separator. For each data source used here, the IPA alignment step is documented in a GitHub repository elsewhere. (While I don't *think* any script here depends on use of Unicode IPA symbols, I haven't - and won't - test that idea, and it is *absolutely required* that contiguous sequences of segments be separated by `.` in data files.)
 - Where language models and n-gram contexts (drawn from speech corpora) are referenced, each of these is assumed to have come from as is from other GitHub repositories.

# Step 1: Make transcribed lexicons

 - Each transcribed lexicon `LEXNAME` should be in a folder (e.g. `LTR_LEXNAME`) containing a file `LTR_LEXNAME.tsv`. For documentation purposes, the source file and a notebook documenting the production of the `.tsv` file should, if practicable be included in the folder as well.
   - A transcribed lexicon `LTR_....tsv` file should have two columns: `Orthographic_Wordform` and `Transcription`.
   - NB: The `LTR_` prefix on transcribed lexicon data files and containing folders is simply a convention for organization and readability, but is not required or expected by any file or script in this repository.

## Check for transcribed lexicons

The assertions in the code below will only succeed if step 1 is complete for all transcribed lexicons listed for checking below.

In [49]:
newdic_destressed_ltr_folder = 'LTR_newdic_destressed'
cmu_destressed_ltr_folder = 'LTR_CMU_destressed'
cmu_stressed_ltr_folder = 'LTR_CMU_stressed'
buckeye_ltr_folder = 'LTR_Buckeye'
# nxt_swbd_ltr_folder = 

LTR_folders = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, cmu_stressed_ltr_folder, buckeye_ltr_folder)
LTR_folders_to_process = (newdic_destressed_ltr_folder, cmu_destressed_ltr_folder, buckeye_ltr_folder)

for dirname in tqdm(LTR_folders_to_process):
    assert path.exists(dirname), 'Transcribed lexicon directory {0} not found in repo directory'.format(dirname)
    fname = path.join(dirname, dirname + '.tsv')
    assert path.exists(fname), 'Transcribed lexicon {0} not found in repo directory'.format(fname)

100%|██████████| 3/3 [00:00<00:00, 2184.15it/s]


## How are transcribed lexicon relations made?

See the `Making a Transcribed Lexicon relation - <LEXNAME>.ipynb` notebook in each `LTR_LEXNAME` folder:

In [51]:
for dirname in LTR_folders_to_process:
    listdir(dirname)

['LTR_newdic_destressed.tsv',
 'Making a Transcribed Lexicon Relation - newdic_destressed.ipynb',
 'newdic_IPA.tsv',
 '.ipynb_checkpoints']

['Making a Transcribed Lexicon Relation - CMU_destressed.ipynb',
 'LTR_CMU_destressed.tsv',
 '.ipynb_checkpoints',
 'cmudict-0.7b_IPA_destressed.tsv']

['LTR_Buckeye.tsv',
 'buckeye_words_analysis_relation.json',
 '.ipynb_checkpoints',
 'Making a Transcribed Lexicon Relation - Buckeye.ipynb']

# Step 2: Segment inventory alignment

## Step 2a: Define inventory alignment projections

The segment inventory of any given transcribed lexicon and the segment inventory of the gating data often do not line up. For the gating data to be usefully applied to a given lexicon of transcriptions, the strings in the lexicon must contain only segments found in the gating data stimuli inventory.

To ensure this happens, the notebook `Gating Data - Transcription Lexicon Alignment Maker.ipynb` 
 - takes as inputs 
     - a transcribed lexicon file path and a gating data file path
     - a lexicon projection file path and a gating data projection file path
 - identifies the inventories of each and what symbols are relatively unique to the lexicon and the gating data
 - produces 
   - *a Jupyter notebook* for **you to open and finish by defining two projection functions** (i.e. Python dictionaries) to be applied to strings in the transcribed lexicon and to the gating data (one function for each). When you finish doing this (and set an export flag in the notebook to True and run the remainder of the notebook), this notebook will produce
     - two *.json files storing these projections* according to the previously provided output file paths.

The cell below will clear all existing alignment folders created using the code in this subsection:

In [9]:
%rm -rf *_aligned_w_*
%rm -rf *" alignment definition"*

The cell below will only succeed if the American English gating data of Warner, McQueen, and Cutler (2014) is contained in the repo directory with a particular directory and filename.

In [10]:
gating_data_folder = 'GD_AmE'
gating_data_fn = 'AmE-diphones-IPA-annotated-columns.csv'
gating_data_fp = path.join(gating_data_folder, gating_data_fn)

assert path.exists(gating_data_folder), 'AmE gating data folder {0} not found in repo directory'.format(gating_data_folder)
assert path.exists(gating_data_fp), 'AmE gating data {0} not found in repo directory'.format(gating_data_fp)

The third cell below will create a notebook for alignment projection definitions for each of the transcribed lexicons from the previous step and the AmE gating data.

In [11]:
def removeExtension(fp):
    dir_name = path.dirname(fp)
    file_name = path.basename(fp)
    ext = file_name.split('.')[-1]
    rest = '.'.join( file_name.split('.')[:-1] )
    return path.join(dir_name, rest)

In [12]:
arg_bundles = []
for LTR_dirname in tqdm(LTR_folders_to_process):
    LTR_fn = LTR_dirname + '.tsv'
    LTR_fp = path.join(LTR_dirname, LTR_fn)
    
    nb_output_name = 'GD_AmE-diphones - ' + LTR_dirname + ' alignment definition' + '.ipynb'
    my_g = gating_data_fp
    my_l = LTR_fp
    my_s = 'destressed'
    
    gd_alignment_dn = 'GD_AmE_' + my_s + '_' + 'aligned_w_' + LTR_dirname
    gd_alignment_fn = 'alignment_of_' + removeExtension(gating_data_fn) + '_w_' + LTR_dirname + '.json'
    gd_alignment_fp = path.join(gd_alignment_dn, gd_alignment_fn)
    if not path.exists(gd_alignment_dn):
        makedirs(gd_alignment_dn)
    my_gp = gd_alignment_fp
    
    ltr_alignment_dn = LTR_dirname + '_aligned_w_' + 'GD_AmE_' + my_s
    ltr_alignment_fn = 'alignment_of_' + LTR_dirname + '_w_' + removeExtension(gating_data_fn) + '.json'
    ltr_alignment_fp = path.join(ltr_alignment_dn, ltr_alignment_fn)
    if not path.exists(ltr_alignment_dn):
        makedirs(ltr_alignment_dn)
    my_lp = ltr_alignment_fp
    
    
    my_arg_bundle = OrderedDict({
        'LTR_dirname':LTR_dirname,
        'LTR_fn':LTR_fn,
        'LTR_fp':LTR_fp,
        'gd_alignment_dn':gd_alignment_dn,
        'gd_alignment_fn':gd_alignment_fn,
        'gd_alignment_fp':gd_alignment_fp,
        'ltr_alignment_dn':ltr_alignment_dn,
        'ltr_alignment_fn':ltr_alignment_fn,
        'ltr_alignment_fp':ltr_alignment_fp,
        'align_def_nb_output_name':nb_output_name,
        'my_g':my_g,
        'my_l':my_l,
        'my_s':my_s,
        'my_gp':my_gp,
        'my_lp':my_lp,
    })
    arg_bundles.append(my_arg_bundle)

100%|██████████| 3/3 [00:00<00:00, 72.22it/s]


In [13]:
for arg_bundle in tqdm(arg_bundles):
    nb = pm.execute_notebook(
        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
        arg_bundle['align_def_nb_output_name'],
        parameters=dict(g = arg_bundle['my_g'], 
                        l = arg_bundle['my_l'], 
                        s = arg_bundle['my_s'], 
                        gp = arg_bundle['my_gp'], 
                        lp = arg_bundle['my_lp'])
    )
#     pm.execute_notebook(
#        'Gating Data - Transcription Lexicon Alignment Maker.ipynb',
#        nb_output_name,
#        parameters=dict(g = my_g, l = my_l, s = my_s, gp = my_gp, lp = my_lp)
#     )
    print("Finished creating alignment definition notebook {0}.\nOpen and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').\n".format(arg_bundle['align_def_nb_output_name']))

  0%|          | 0/3 [00:00<?, ?it/s]Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb

 33%|███▎      | 1/3 [00:10<00:20, 10.05s/it]s]

Finished creating alignment definition notebook GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb

 67%|██████▋   | 2/3 [00:23<00:10, 10.93s/it]s]

Finished creating alignment definition notebook GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



Input Notebook:  Gating Data - Transcription Lexicon Alignment Maker.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb

100%|██████████| 3/3 [00:34<00:00, 11.02s/it]s]

Finished creating alignment definition notebook GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb.
Open and run the notebook, complete the projection definition, and run the remainder of the notebook (remembering to change the export flag to 'True').



## Step 2b: Apply inventory alignment projections

The cell below will clear all existing alignment folders created using the code in this subsection:

In [14]:
%rm -rf *" alignment application "*

### Check for projection definitions

The cell below will succeed if you have run each of the previously produced notebooks correctly and produced a projection mapping file.

In [15]:
for arg_bundle in tqdm(arg_bundles):
    args = arg_bundle
    assert path.exists(args['gd_alignment_fp']), 'Gating data alignment projection mapping not found:\n\t{0}'.format(args['gd_alignment_fp'])
    assert path.exists(args['ltr_alignment_fp']), 'Transcribed lexicon data alignment projection mapping not found:\n\t{0}'.format(args['ltr_alignment_fp'])

100%|██████████| 3/3 [00:00<00:00, 1466.88it/s]


### How are inventory alignment projections actually applied?

See `Align transcriptions.ipynb`.

### Apply projection definitions

The cell below applies each pair of alignment projections to each matched pair of gating data and transcribed lexicon choice:

In [39]:
for arg_bundle in arg_bundles:
    args = arg_bundle
    LTR_fn = args['LTR_fn']
    
    my_pg = args['my_gp']
    my_g = args['my_g']
    my_o_fn = 'GD_AmE-diphones' + '_aligned_w_' + removeExtension(LTR_fn) + '.tsv'
    my_og = path.join(args['gd_alignment_dn'], my_o_fn)
    args['align_apply_gd_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + 'AmE-diphones' + '.ipynb'
    args['my_og'] = my_og
    print("Creating notebook '{0}' w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}".format(args['align_apply_gd_nb_output_name'], my_pg, my_g, my_og))
    nb = pm.execute_notebook(
        'Align transcriptions.ipynb',
        args['align_apply_gd_nb_output_name'],
        parameters=dict(p = my_pg,
                        g = my_g,
                        o = my_og)
    )
    print('Finished applying alignment projection\n\tp = {0}\nto\n\tg = {1}\nResult saved to\n\t{2}'.format(my_pg, my_g, my_og))
    print(' ')
    
    my_pl = args['my_lp']
    my_l = args['my_l']
    my_o_fn = removeExtension(LTR_fn) + '_aligned_w_' + 'GD_AmE-diphones' + '.tsv'
    my_ol = path.join(args['ltr_alignment_dn'], my_o_fn)
    args['align_apply_ltr_nb_output_name'] = 'GD_AmE-diphones - ' + removeExtension(LTR_fn) + ' alignment application to ' + removeExtension(LTR_fn) + '.ipynb'
    print('Creating notebook {0} w/ args p, g, o = \n\t{1}\n\t{2}\n\t{3}'.format(args['align_apply_ltr_nb_output_name'], my_pg, my_l, my_ol))
    args['my_ol'] = my_ol
    nb = pm.execute_notebook(
        'Align transcriptions.ipynb',
        args['align_apply_ltr_nb_output_name'],
        parameters=dict(p = my_pl,
                        l = my_l,
                        o = my_ol)
    )
    print('Finished applying alignment projection\n\tp = {0}\nto\n\tl = {1}\nResult saved to\n\t{2}'.format(my_pl, my_l, my_ol))
    print('\n')

Creating notebook 'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb
100%|██████████| 64/64 [00:07<00:00,  8.10it/s]


Finished applying alignment projection
	p = GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
to
	g = GD_AmE/AmE-diphones-IPA-annotated-columns.csv
Result saved to
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv
 
Creating notebook GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_newdic_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json
	LTR_newdic_destressed/LTR_newdic_destressed.tsv
	LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb
100%|██████████| 64/64 [00:02<00:00, 23.18it/s]


Finished applying alignment projection
	p = LTR_newdic_destressed_aligned_w_GD_AmE_destressed/alignment_of_LTR_newdic_destressed_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_newdic_destressed/LTR_newdic_destressed.tsv
Result saved to
	LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_w_GD_AmE-diphones.tsv


Creating notebook 'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_CMU_destressed/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_CMU_destressed.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv


...

Finished applying alignment projection
	p = LTR_CMU_destressed_aligned_w_GD_AmE_destressed/alignment_of_LTR_CMU_destressed_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_CMU_destressed/LTR_CMU_destressed.tsv
Result saved to
	LTR_CMU_destressed_aligned_w_GD_AmE_destressed/LTR_CMU_destressed_aligned_w_GD_AmE-diphones.tsv


Creating notebook 'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb' w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
	GD_AmE/AmE-diphones-IPA-annotated-columns.csv
	GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb
100%|██████████| 64/64 [00:07<00:00,  8.24it/s]


Finished applying alignment projection
	p = GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
to
	g = GD_AmE/AmE-diphones-IPA-annotated-columns.csv
Result saved to
	GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv
 
Creating notebook GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb w/ args p, g, o = 
	GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json
	LTR_Buckeye/LTR_Buckeye.tsv
	LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv


Input Notebook:  Align transcriptions.ipynb
Output Notebook: GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb
100%|██████████| 64/64 [00:05<00:00, 12.16it/s]


Finished applying alignment projection
	p = LTR_Buckeye_aligned_w_GD_AmE_destressed/alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json
to
	l = LTR_Buckeye/LTR_Buckeye.tsv
Result saved to
	LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_w_GD_AmE-diphones.tsv




# Step 3: Generating channel and lexicon distributions

I. **Channel distribution**

 A. Given gating data, we can define uniphone and triphone channel distributions.
 
II. **Lexicon distribution**

A. **Alignment of the vocabulary with the channel model and with a language model, alignment of the channel model with a lexicon**
 1. Given a transcribed lexicon relation and information about the conditioning events and sample space of a channel model, we can project a transcribed lexicon relation down to just what can be modeled using the channel model.
 2. Given the vocabulary of an (orthographic) $n$-gram model, we can project a transcribed lexicon relation down to just what can be aligned with the vocabulary of the $n$-gram model. (I.e. if the $n$-gram model vocabulary is a unary relation on orthographic wordforms, this calculates the *natural join* of the $n$-gram model vocabulary and the transcribed lexicon relation.)
 3. Given a channel model and a transcribed lexicon relation that the channel model will be applied to, we can remove $n$-phones from the channel model that do not appear in the transcribed lexicon relation.

B. **Distribution construction**
 1. Given a transcribed lexicon relation, we can define a family of conditional distributions on segmental wordforms given a choice of orthographic wordform: e.g. if orthographic wordform $w$ can surface as all and only the segmental sequences $s_0$ and $s_1$, then we construct a uniform distribution over $s_0$ and $s_1$ given $w$.
 2. Given a (unigram) distribution over orthographic wordforms and orthographic->segmental wordform conditional distributions, we can define a distribution over segmental wordforms.
 3. Given an $n$-gram model, the vocabulary of the model, a set of (orthographic) $n$-gram contexts $C$, and a target vocabulary of orthographic wordforms of interest (i.e. the ones we have transcriptions for), we can calculate a distribution over the target vocabulary for each $n$-gram context.

In [40]:
arg_bundle

OrderedDict([('LTR_dirname', 'LTR_Buckeye'),
             ('LTR_fn', 'LTR_Buckeye.tsv'),
             ('LTR_fp', 'LTR_Buckeye/LTR_Buckeye.tsv'),
             ('gd_alignment_dn', 'GD_AmE_destressed_aligned_w_LTR_Buckeye'),
             ('gd_alignment_fn',
              'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json'),
             ('gd_alignment_fp',
              'GD_AmE_destressed_aligned_w_LTR_Buckeye/alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json'),
             ('ltr_alignment_dn', 'LTR_Buckeye_aligned_w_GD_AmE_destressed'),
             ('ltr_alignment_fn',
              'alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json'),
             ('ltr_alignment_fp',
              'LTR_Buckeye_aligned_w_GD_AmE_destressed/alignment_of_LTR_Buckeye_w_AmE-diphones-IPA-annotated-columns.json'),
             ('align_def_nb_output_name',
              'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'),
             ('my_g', 'GD_AmE

## Step 3a: Generating channel distributions and associated metadata

In [23]:
%ls -d GD_*

 GD_AmE/
 GD_AmE_destressed_aligned_w_LTR_Buckeye/
 GD_AmE_destressed_aligned_w_LTR_CMU_destressed/
 GD_AmE_destressed_aligned_w_LTR_newdic_destressed/
'GD_AmE-diphones - LTR_Buckeye alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment application to LTR_Buckeye.ipynb'
'GD_AmE-diphones - LTR_Buckeye alignment definition.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment application to LTR_CMU_destressed.ipynb'
'GD_AmE-diphones - LTR_CMU_destressed alignment definition.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to AmE-diphones.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment application to LTR_newdic_destressed.ipynb'
'GD_AmE-diphones - LTR_newdic_destressed alignment definition.ipynb'


In [27]:
gating_data_folders = ('GD_AmE', ) + tuple(map(lambda ab: ab['gd_alignment_dn'], arg_bundles))
gating_data_folders

('GD_AmE',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye')

In [44]:
gating_data_fps = ('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',) + \
                  tuple(map(lambda ab: ab['my_og'], arg_bundles))
gating_data_fps

('GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv')

### Metadata

First (for downstream convenience) we identify the $n$-phones (not) contained in and (not) constructible from each of the versions of the gating data.

**NB:** This is done by calling the notebook `Run n-phone analysis of gating data.ipynb`, passing it **the filepath to a gating data `.csv`/`.tsv` file** and **a path to an output directory** for the dozen or so files the notebook will produce.

In [46]:
for gating_data_fp in gating_data_fps:
    gd_dir = path.dirname(gating_data_fp)
    nb = pm.execute_notebook(
        'Run n-phone analysis of gating data.ipynb',
#         args['align_apply_ltr_nb_output_name'],
        path.join(gd_dir, gd_dir) + " n-phone analysis.ipynb",
        parameters=dict(g = gating_data_fp,
                        o = gd_dir)
    )
    listdir(gd_dir)
    print('\n')

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE/GD_AmE n-phone analysis.ipynb
100%|██████████| 43/43 [00:26<00:00,  1.60it/s]


['destressed response diphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed stimuli uniphones.txt',
 'destressed stimuli illegal diphones.txt',
 'AmE-diphones-IPA-annotated-columns.csv',
 'destressed response illegal diphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli diphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 '.ipynb_checkpoints',
 'destressed response diphone-based illegal triphones.txt',
 'GD_AmE n-phone analysis.ipynb']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_newdic_destressed/GD_AmE_destressed_aligned_w_LTR_newdic_destressed n-phone analysis.ipynb
100%|██████████| 43/43 [00:27<00:00,  1.58it/s]


['destressed stimuli uniphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_newdic_destressed n-phone analysis.ipynb',
 'destressed response uniphones.txt',
 'destressed stimuli illegal diphones.txt',
 'destressed response diphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'GD_AmE-diphones_aligned_w_LTR_newdic_destressed.tsv',
 'destressed response illegal diphones.txt',
 'destressed stimuli diphones.txt',
 'stressed stimuli uniphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'destressed response diphone-based constructible triphones.txt',
 'stressed stimuli illegal diphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_newdic_destressed.json']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_CMU_destressed/GD_AmE_destressed_aligned_w_LTR_CMU_destressed n-phone analysis.ipynb
100%|██████████| 43/43 [00:27<00:00,  1.56it/s]


['destressed response illegal diphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'destressed stimuli illegal diphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'stressed stimuli diphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli illegal diphones.txt',
 'GD_AmE_destressed_aligned_w_LTR_CMU_destressed n-phone analysis.ipynb',
 'GD_AmE-diphones_aligned_w_LTR_CMU_destressed.tsv',
 'destressed stimuli diphones.txt',
 'destressed stimuli uniphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_CMU_destressed.json',
 'destressed response diphone-based constructible triphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed response diphones.txt',
 'stressed stimuli uniphones.txt']

Input Notebook:  Run n-phone analysis of gating data.ipynb
Output Notebook: GD_AmE_destressed_aligned_w_LTR_Buckeye/GD_AmE_destressed_aligned_w_LTR_Buckeye n-phone analysis.ipynb
100%|██████████| 43/43 [00:26<00:00,  1.64it/s]


['destressed response diphone-based constructible triphones.txt',
 'destressed stimuli diphone-based constructible triphones.txt',
 'destressed stimuli diphone-based illegal triphones.txt',
 'destressed response illegal diphones.txt',
 'destressed stimuli illegal diphones.txt',
 'stressed stimuli diphone-based illegal triphones.txt',
 'destressed response uniphones.txt',
 'stressed stimuli diphone-based constructible triphones.txt',
 'destressed stimuli diphones.txt',
 'destressed response diphone-based illegal triphones.txt',
 'stressed stimuli uniphones.txt',
 'destressed response diphones.txt',
 'destressed stimuli uniphones.txt',
 'stressed stimuli diphones.txt',
 'alignment_of_AmE-diphones-IPA-annotated-columns_w_LTR_Buckeye.json',
 'GD_AmE-diphones_aligned_w_LTR_Buckeye.tsv',
 'GD_AmE_destressed_aligned_w_LTR_Buckeye n-phone analysis.ipynb',
 'stressed stimuli illegal diphones.txt']

### Channel distributions

Next, the notebook `Producing channel distributions.ipynb` will create `.json` files defining (among other things) a uniphone and triphone channel distribution. It requires the following arguments to specify information about what kind of channel model to build and where to put it:
 - **a filepath** to a gating data file
 - **a directory** containing metadata indicating possible/impossible $n$-phones
 - **a string argument** ("stressed" or "destressed") indicating whether the distribution will be over a segment inventory with or without stress information
 - **a real valued**  smoothing parameter (a pseudocount to add to every channel outcome)
 - **an output directory** to write the channel model to.

In [53]:
pseudocounts = (0, 0.01, 0.05, 0.1)

In [71]:
cm_arg_bundles = []
for gating_data_fp in gating_data_fps:
    metadata_dir = path.dirname(gating_data_fp)
    s = "destressed"
    channel_model_dir_stem = 'CM' + metadata_dir[2:]
    
    for pc in pseudocounts:
        channel_model_dir_suffix = '_pseudocount' + str(pc)
        if metadata_dir == 'GD_AmE':
            channel_model_dir = channel_model_dir_stem + '_' + s + '_unaligned' + '_' + channel_model_dir_suffix
        else:
            channel_model_dir = channel_model_dir_stem + channel_model_dir_suffix
        nb_output_name = 'Producing channel distributions from ' + metadata_dir + ', pc={0}'.format(pc) + '.ipynb'
        new_arg_bundle = {'gating_data_fp':gating_data_fp,
                          'metadata_dir':metadata_dir,
                          's':s,
                          'c':pc,
                          'cm_dir':channel_model_dir,
                          'nb_output_name':nb_output_name}
        cm_arg_bundles.append(new_arg_bundle)

In [70]:
cm_arg_bundles[0]

{'gating_data_fp': 'GD_AmE/AmE-diphones-IPA-annotated-columns.csv',
 'metadata_dir': 'GD_AmE',
 's': 'destressed',
 'c': 0,
 'cm_dir': 'CM_AmE_pseudocount0',
 'nb_output_name': 'Producing channel distributions from GD_AmE, pc=0.ipynb'}

In [ ]:
for ab in cm_arg_bundles:
    if not path.exists(ab['cm_dir']):
        mkdir(ab['cm_dir'])
    
    nb = pm.execute_notebook(
    'Producing channel distributions.ipynb',
    path.join(ab['cm_dir'], ab['nb_output_name']),
    parameters=dict(g = ab['gating_data_fp'],
                    m = ab['metadata_dir'],
                    s = ab['s'],
                    c = ab['c'],
                    o = ab['cm_dir'])
    )
    print('\n')

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_unaligned_destressed_pseudocount0/Producing channel distributions from GD_AmE, pc=0.ipynb






  0%|          | 0/189 [00:00<?, ?it/s]





  1%|          | 1/189 [00:00<00:19,  9.78it/s]





  2%|▏         | 3/189 [00:00<00:18, 10.25it/s]





  3%|▎         | 5/189 [00:00<00:18, 10.12it/s]





  4%|▎         | 7/189 [00:00<00:17, 10.41it/s]





  4%|▍         | 8/189 [00:00<00:17, 10.18it/s]





  5%|▍         | 9/189 [00:00<00:18,  9.64it/s]





  6%|▌         | 11/189 [00:01<00:16, 10.66it/s]





  7%|▋         | 13/189 [00:01<00:15, 11.15it/s]





  8%|▊         | 15/189 [00:01<00:15, 10.99it/s]





  9%|▉         | 17/189 [00:01<00:15, 11.01it/s]





 10%|█         | 19/189 [00:02<00:27,  6.16it/s]





 11%|█         | 20/189 [00:02<00:24,  6.95it/s]





 11%|█         | 21/189 [00:02<00:24,  6.73it/s]





 12%|█▏        | 23/189 [00:02<00:22,  7.31it/s]





 13%|█▎        | 25/189 [00:02

Input Notebook:  Producing channel distributions.ipynb
Output Notebook: CM_AmE_unaligned_destressed_pseudocount0.01/Producing channel distributions from GD_AmE, pc=0.01.ipynb






  0%|          | 0/189 [00:00<?, ?it/s]





  1%|          | 1/189 [00:00<00:25,  7.29it/s]





  1%|          | 2/189 [00:00<00:25,  7.42it/s]





  2%|▏         | 4/189 [00:00<00:23,  7.86it/s]





  3%|▎         | 5/189 [00:00<00:23,  7.93it/s]





  3%|▎         | 6/189 [00:00<00:22,  8.24it/s]





  4%|▍         | 8/189 [00:00<00:20,  8.85it/s]





  5%|▌         | 10/189 [00:01<00:20,  8.89it/s]





  6%|▋         | 12/189 [00:01<00:18,  9.43it/s]





  7%|▋         | 13/189 [00:01<00:19,  9.11it/s]





  7%|▋         | 14/189 [00:01<00:23,  7.42it/s]





  8%|▊         | 16/189 [00:01<00:20,  8.35it/s]





 10%|▉         | 18/189 [00:01<00:18,  9.09it/s]





 10%|█         | 19/189 [00:02<00:57,  2.98it/s]





 11%|█         | 20/189 [00:02<00:44,  3.77it/s]





 12%|█▏        | 22/189 

## Step 3b: Filtering and alignment ('vocabulary normalization')

### Filter transcription lexicons to only include words that can be modeled by a given channel distribution

### Filter transcription lexicons to only include words that are in a language model's vocabulary

### Filter the conditioning events of channel distributions to only include $n$-phones contained in a transcription lexicon's segmental wordform list

## Step 3c: Define conditional distributions on segmental wordforms given an orthographic wordform

## Step 3d: Define a distribution over the transcribed vocabulary of a language model for each n-gram context in a set of (possibly empty) n-gram contexts

# Step 4: Generating posterior distributions

# Step 5: Generating analysis measures